# Exploring the Orbital Environment -- Live TLE Data from CelesTrak

The CDM dataset gives us historical conjunction events to train on. But for the actual application, I want to visualize the current state of objects in orbit. CelesTrak provides free, real-time Two-Line Element (TLE) data for every tracked object -- no API key needed.

TLEs encode an object's orbit in a compact format. From a TLE, you can compute the object's position at any point in time using a propagation algorithm called SGP4. This is what satellite tracking apps use under the hood.

Let me pull some live data and see what the orbital environment looks like right now.

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

TLE_DIR = Path('../data/tle')

In [ ]:
# Check what TLE files we have (from fetch_tles.py)
tle_files = list(TLE_DIR.glob('*.json'))
print(f'TLE files available: {len(tle_files)}')
for f in sorted(tle_files):
    print(f'  {f.name}')

In [ ]:
# If we don't have local files yet, fetch directly from CelesTrak
def fetch_tle_group(group):
    url = f'https://celestrak.org/NORAD/elements/gp.php?GROUP={group}&FORMAT=json'
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    return resp.json()

# Load or fetch active satellites
active_file = TLE_DIR / 'active.json'
if active_file.exists():
    with open(active_file) as f:
        active_sats = json.load(f)
    print(f'Loaded {len(active_sats)} active satellites from cache')
else:
    print('Fetching active satellites from CelesTrak...')
    active_sats = fetch_tle_group('active')
    print(f'Got {len(active_sats)} active satellites')

# Convert to DataFrame for easier analysis
df_active = pd.DataFrame(active_sats)
df_active.head()

In [ ]:
print(f'Columns: {list(df_active.columns)}')
print(f'\nShape: {df_active.shape}')
print(f'\nData types:')
print(df_active.dtypes)

## Understanding TLE parameters

Each object has these orbital elements:

- **MEAN_MOTION**: revolutions per day (higher = lower orbit = faster)
- **ECCENTRICITY**: how circular the orbit is (0 = perfect circle, closer to 1 = more elliptical)
- **INCLINATION**: angle of the orbital plane relative to the equator (0 = equatorial, 90 = polar)
- **RA_OF_ASC_NODE (RAAN)**: orientation of the orbital plane around Earth's axis
- **ARG_OF_PERICENTER**: where the lowest point of the orbit is within the orbital plane
- **MEAN_ANOMALY**: where the satellite is in its orbit right now
- **BSTAR**: drag coefficient (captures atmospheric drag effects)

From mean motion, we can compute altitude. The relationship is:
- Mean motion (n) in rev/day relates to semi-major axis (a) by Kepler's third law
- Altitude = a - Earth_radius

In [ ]:
# Compute altitude from mean motion using Kepler's third law
# n = rev/day, convert to rad/s first
# a = (mu / n^2)^(1/3) where mu = 3.986e14 m^3/s^2

EARTH_RADIUS_KM = 6371.0
MU = 3.986004418e14  # m^3/s^2

def mean_motion_to_altitude(n_rev_per_day):
    """Convert mean motion (rev/day) to approximate altitude (km)."""
    n_rad_per_sec = n_rev_per_day * 2 * np.pi / 86400.0
    a_meters = (MU / (n_rad_per_sec ** 2)) ** (1/3)
    a_km = a_meters / 1000.0
    altitude_km = a_km - EARTH_RADIUS_KM
    return altitude_km

df_active['altitude_km'] = mean_motion_to_altitude(df_active['MEAN_MOTION'])

print('Altitude statistics (km):')
print(df_active['altitude_km'].describe())

# Quick sanity check -- ISS should be around 400-420 km
iss = df_active[df_active['OBJECT_NAME'].str.contains('ISS', case=False, na=False)]
if len(iss) > 0:
    print(f'\nISS altitude: {iss.iloc[0]["altitude_km"]:.1f} km (should be ~410-420)')

## Altitude distribution -- where is everything orbiting?

This is probably the most important plot for understanding the debris problem. Certain altitude bands are way more crowded than others, and crowded = dangerous.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram of altitudes (LEO focus)
leo_mask = df_active['altitude_km'] < 2000
axes[0].hist(df_active.loc[leo_mask, 'altitude_km'], bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Altitude (km)')
axes[0].set_ylabel('Number of Objects')
axes[0].set_title('Active Satellite Altitude Distribution (LEO < 2000 km)')
axes[0].axvline(x=550, color='red', linestyle='--', alpha=0.7, label='Starlink shell (~550 km)')
axes[0].axvline(x=340, color='orange', linestyle='--', alpha=0.7, label='OneWeb (~340 km)')
axes[0].legend()

# Full range including GEO
axes[1].hist(df_active['altitude_km'].clip(upper=40000), bins=100, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('Altitude (km)')
axes[1].set_ylabel('Number of Objects')
axes[1].set_title('Full Altitude Range (includes GEO at ~35,786 km)')
axes[1].axvline(x=35786, color='purple', linestyle='--', alpha=0.7, label='GEO belt')
axes[1].legend()

plt.tight_layout()
plt.show()

# What percentage is in LEO?
pct_leo = leo_mask.mean() * 100
print(f'{pct_leo:.1f}% of active satellites are in LEO (< 2000 km)')

## Inclination distribution

Inclination tells us the angle of the orbit relative to the equator. Certain inclinations are popular because they serve specific purposes:
- ~0 degrees: equatorial (GEO communication satellites)
- ~51.6 degrees: ISS orbit
- ~53 degrees: Starlink
- ~87-98 degrees: sun-synchronous (Earth observation, weather)
- 90 degrees: polar

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(df_active['INCLINATION'], bins=180, edgecolor='black', alpha=0.7, color='teal')
ax.set_xlabel('Inclination (degrees)')
ax.set_ylabel('Number of Objects')
ax.set_title('Orbital Inclination Distribution')

# Annotate popular inclinations
annotations = [
    (0, 'Equatorial/GEO'),
    (53, 'Starlink'),
    (97.5, 'Sun-synchronous'),
]
for inc, label in annotations:
    ax.axvline(x=inc, color='red', linestyle='--', alpha=0.5)
    ax.annotate(label, xy=(inc, ax.get_ylim()[1]*0.9), fontsize=8,
                ha='center', color='red')

plt.tight_layout()
plt.show()

## Now let's look at debris

The active satellite catalog is only part of the picture. The really scary stuff is the debris -- fragments from collisions and ASAT (anti-satellite weapon) tests. Let me load the debris groups and compare.

In [ ]:
# Load debris data from our local cache or fetch from CelesTrak
debris_groups = {
    'cosmos-2251-debris': 'Cosmos 2251 (2009 collision)',
    'iridium-33-debris': 'Iridium 33 (2009 collision)',
    'fengyun-1c-debris': 'Fengyun 1C (2007 ASAT test)',
    'cosmos-1408-debris': 'Cosmos 1408 (2021 ASAT test)',
}

all_debris = []
for group, label in debris_groups.items():
    local_file = TLE_DIR / f'{group}.json'
    if local_file.exists():
        with open(local_file) as f:
            data = json.load(f)
    else:
        try:
            data = fetch_tle_group(group)
        except Exception as e:
            print(f'Could not fetch {group}: {e}')
            continue
    
    df_debris = pd.DataFrame(data)
    df_debris['source'] = label
    df_debris['altitude_km'] = mean_motion_to_altitude(df_debris['MEAN_MOTION'])
    all_debris.append(df_debris)
    print(f'{label}: {len(df_debris)} tracked fragments')

if all_debris:
    df_debris_all = pd.concat(all_debris, ignore_index=True)
    print(f'\nTotal tracked debris fragments: {len(df_debris_all)}')
else:
    print('No debris data available yet. Run scripts/fetch_tles.py first.')

In [ ]:
# Compare altitude distributions of different debris sources
if len(all_debris) > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for source in df_debris_all['source'].unique():
        subset = df_debris_all[df_debris_all['source'] == source]
        ax.hist(subset['altitude_km'].clip(upper=2000), bins=80, alpha=0.5,
                label=f'{source} ({len(subset)} fragments)')
    
    ax.set_xlabel('Altitude (km)')
    ax.set_ylabel('Number of Debris Fragments')
    ax.set_title('Debris Altitude Distribution by Source Event')
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Which altitude band has the most debris?
    print('\nDebris density by altitude band:')
    bins = pd.cut(df_debris_all['altitude_km'], bins=range(200, 1600, 50))
    print(bins.value_counts().head(10))

## Eccentricity -- how circular are these orbits?

Most LEO orbits are nearly circular (eccentricity close to 0). But debris from explosions or collisions can get kicked into eccentric orbits, which makes them harder to predict and more likely to cross paths with other objects.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Active satellites
axes[0].hist(df_active['ECCENTRICITY'].clip(upper=0.1), bins=100, 
             edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Eccentricity')
axes[0].set_ylabel('Count')
axes[0].set_title('Active Satellites - Eccentricity')

# Debris
if len(all_debris) > 0:
    axes[1].hist(df_debris_all['ECCENTRICITY'].clip(upper=0.1), bins=100,
                 edgecolor='black', alpha=0.7, color='red')
    axes[1].set_xlabel('Eccentricity')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Debris Fragments - Eccentricity')

plt.tight_layout()
plt.show()

print(f'Active sats - median eccentricity: {df_active["ECCENTRICITY"].median():.6f}')
if len(all_debris) > 0:
    print(f'Debris - median eccentricity: {df_debris_all["ECCENTRICITY"].median():.6f}')

## Altitude vs inclination scatter -- the full picture

This is the "God's eye view" of the orbital environment. Each dot is a tracked object. You can immediately see the popular orbit regimes and where the crowded zones are.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

# Plot active satellites
ax.scatter(df_active['INCLINATION'], df_active['altitude_km'].clip(upper=40000),
           s=1, alpha=0.3, c='steelblue', label=f'Active satellites ({len(df_active)})')

# Overlay debris
if len(all_debris) > 0:
    ax.scatter(df_debris_all['INCLINATION'], df_debris_all['altitude_km'].clip(upper=40000),
               s=2, alpha=0.5, c='red', label=f'Debris ({len(df_debris_all)})')

ax.set_xlabel('Inclination (degrees)', fontsize=12)
ax.set_ylabel('Altitude (km)', fontsize=12)
ax.set_title('Orbital Environment: Altitude vs Inclination', fontsize=14)
ax.legend(fontsize=10)

# Annotate key regions
ax.annotate('Starlink\nshells', xy=(53, 550), fontsize=9, ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
ax.annotate('GEO belt', xy=(0, 35786), fontsize=9, ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
ax.annotate('Sun-sync\ncorridor', xy=(97, 700), fontsize=9, ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

plt.tight_layout()
plt.show()

## B-star drag coefficient

BSTAR captures atmospheric drag effects. Objects in lower orbits experience more drag and decay faster. This is actually useful for our project because high-drag objects have more uncertain future positions -- their orbits change faster, making conjunction prediction harder.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

# BSTAR vs altitude -- expect higher drag at lower altitudes
mask = (df_active['altitude_km'] < 2000) & (df_active['BSTAR'].abs() < 0.01)
ax.scatter(df_active.loc[mask, 'altitude_km'], 
           df_active.loc[mask, 'BSTAR'].abs(),
           s=1, alpha=0.3)
ax.set_xlabel('Altitude (km)')
ax.set_ylabel('|BSTAR| (drag coefficient)')
ax.set_title('Atmospheric Drag vs Altitude (LEO)')
ax.set_yscale('log')
plt.tight_layout()
plt.show()

## Starlink vs the rest of the catalog

SpaceX has launched thousands of Starlink satellites. I'm curious how much of the active catalog they represent and whether they cluster in specific orbital regimes.

In [ ]:
# Load Starlink data separately
starlink_file = TLE_DIR / 'starlink.json'
if starlink_file.exists():
    with open(starlink_file) as f:
        starlink = json.load(f)
    df_starlink = pd.DataFrame(starlink)
    df_starlink['altitude_km'] = mean_motion_to_altitude(df_starlink['MEAN_MOTION'])
    print(f'Starlink satellites: {len(df_starlink)}')
    print(f'Fraction of active catalog: {len(df_starlink)/len(df_active)*100:.1f}%')
    print(f'\nAltitude range: {df_starlink["altitude_km"].min():.0f} - {df_starlink["altitude_km"].max():.0f} km')
    print(f'Inclination range: {df_starlink["INCLINATION"].min():.1f} - {df_starlink["INCLINATION"].max():.1f} degrees')
    
    # Altitude histogram
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.hist(df_starlink['altitude_km'], bins=50, edgecolor='black', alpha=0.7)
    ax.set_xlabel('Altitude (km)')
    ax.set_ylabel('Number of Starlink Satellites')
    ax.set_title(f'Starlink Constellation Altitude Distribution ({len(df_starlink)} satellites)')
    plt.tight_layout()
    plt.show()
else:
    print('No Starlink data cached. Run scripts/fetch_tles.py')
    # Try to identify Starlink in the active catalog by name
    starlink_mask = df_active['OBJECT_NAME'].str.contains('STARLINK', case=False, na=False)
    print(f'Starlink in active catalog (by name): {starlink_mask.sum()}')

## Summary of orbital environment

Key findings:

1. **Most objects are in LEO** (< 2000 km altitude), with massive clustering around 500-600 km (Starlink) and sun-synchronous orbits (~97 degree inclination, 600-800 km)
2. **Starlink dominates the active catalog** -- they're a huge fraction of everything up there
3. **Debris altitude distributions tell a story** -- you can see the specific events (Fengyun, Cosmos/Iridium) as distinct altitude bands
4. **Eccentricity is mostly near-zero** for active sats, slightly higher for debris (collisions kick fragments into eccentric orbits)
5. **Drag is a big deal below ~500 km** -- BSTAR increases dramatically, making predictions harder

For the visualization app, the 3D globe should color-code by object type (active vs debris) and the altitude histogram should be one of the dashboard panels. The density of certain altitude bands is visually striking and immediately communicates the collision risk problem.

For the ML models, altitude band and inclination will be strong features for the naive baseline. The full TLE parameters (eccentricity, RAAN, argument of perigee) will be important for the classical and deep learning models.